# 使用 Mistral LLM 的 AWS DeepRacer 模型评估器

> _此 notebook 可以在 SageMaker Notebook Instance 的 **`conda_python3`** 内核中正常工作_

本次动手实践研讨会演示了如何使用 Amazon Bedrock 结合 Mistral Large（一种大型语言模型 LLM）以及 LangChain 库来构建对话式智能体。该智能体旨在提供有关 AWS DeepRacer 模型和训练的见解和建议。

研讨会展示了如何：

* 创建自定义 LangChain 工具，让智能体能够与 AWS DeepRacer 服务 API 进行交互。这包括列出可用的模型、下载模型工件以及提取模型元数据，如训练数据和奖励函数。

* 在 LangChain 中初始化一个 ReAct 智能体，并为其提供自定义工具。智能体可以根据用户的问题来推理调用哪些工具。

* 使用提示技术，如少样本学习（few-shot learning），通过几个示例来提高智能体的推理能力。

* 如果智能体的响应与预期格式不匹配，可以优雅地处理错误。

* 利用自定义工具，使智能体能够提供有关 AWS DeepRacer 模型的训练数据、超参数、奖励函数等方面的见解。

通过本次动手实践研讨会，与会者将能够使用 LLM 构建对话式智能体，通过自定义接口与 AWS 服务集成。关键要点是能够使用工具扩展智能体的能力、设计模块化智能体以及应用提示技术来改善推理。

## 环境设置

在运行本 notebook 的其余部分之前，您需要安装必要的库，并确保与 Amazon Bedrock 和 AWS DeepRacer 服务 API 建立了有效的连接。

对于本实验，我们需要一些额外的依赖项：
- Boto3，AWS 的 Python SDK
- ChromaDB，用于存储向量嵌入
- LangChain，用于开发由语言模型驱动的应用程序的框架
- PyYAML，YAML 解析器

⚠️ 您将看到 pip 依赖项错误，您可以安全地忽略这些错误。⚠️

忽略错误：pip 的依赖项解析器目前没有考虑所有已安装的包。此行为是以下依赖项冲突的根源。

In [ ]:
# 安装所需的Python包
%pip install --upgrade pip

# Langchain and associated utilities change frequently
# Here, we pin the specific versions we want to use
%pip install langchain-core==0.2.11
%pip install langchain-aws==0.1.9
%pip install langchain==0.2.6
%pip install langchainhub==0.1.20
%pip install langchain_chroma==0.1.2
%pip install chromadb==0.5.3

### 添加工具库

工具库包含用于与 AWS DeepRacer 服务 API 交互、从 DeepRacer 模型中提取相关信息等的代码。

In [ ]:
import os
import sys
import importlib

module_path = "./utils"
sys.path.append(os.path.abspath(module_path))

from utils import print_ww, deepracer_local

# Use importlib to force reloading of local helpfer functions
# (useful for debugging when updating these functions, as it avoids the need
# to restart the notebook kernel)
importlib.reload(deepracer_local)

本 notebook 还需要访问已作为研讨会设置的一部分部署到所使用的 AWS 账户中的 AWS 基础设施。AWS 基础设施的设置是使用 CloudFormation 完成的。

### DeepRacer 服务连接

注意：我们将使用此 Jupyter 笔记本本地存储的模型，而不是直接连接到 DeepRacer 服务。

In [ ]:
###########
# 辅助函数 #
###########
def translation_format(out):
    out = out.strip()

    # Perform some parsing (to remove "Chinese: " from the start of the model's output)
    if out.lower().startswith('chinese:') or out.lower().startswith('中文:'):
        out = ':'.join(out.split(':')[1:]).strip()

    return out

In [ ]:
# 列出可用的模型（来自deepracer_models文件夹）
models = deepracer_local.list_models()
for model in models:
    print(model)

### Bedrock 服务连接

In [ ]:
# 连接到Bedrock
import boto3
bedrock_client = boto3.client('bedrock-runtime')

### LLM 设置

在本实验中，我们将使用以下 LLM：
- Mistral Large，用于推理和行动（ReAct）智能体，以及构建 DeepRacer 问答 LLM 工具。
- Amazon Titan 嵌入模型，允许 DeepRacer 问答 LLM 工具在 DeepRacer 文档上进行语义搜索，以提高响应质量。

In [ ]:
# 配置 Mistral (LLM) 和 Titan（Embeddings）
from langchain_aws import BedrockLLM
from langchain_aws import BedrockEmbeddings

bedrock_llm_client = BedrockLLM(
    model_id="mistral.mixtral-8x7b-instruct-v0:1",
    #model_id="mistral.mistral-large-2402-v1:0",
    client=bedrock_client,
    model_kwargs={"temperature": 0},
)

bedrock_embeddings_client = BedrockEmbeddings(
    client=bedrock_client,
    model_id="amazon.titan-embed-text-v1"
)

In [ ]:
# 创建一个新链(LangChain的chain)，用于提供从英语到中文的翻译
from langchain_core.prompts import PromptTemplate

template = '''
You are an expert translation bot, with a vast knowledge of human languages.
Your task is to translate English content after <<<>>> into Chinese.

###
Here are some examples:

English: Hello, friend
Chinese: 你好朋友

English: Today, we will learn about DeepRacer
Chinese: 今天，我们来学习下DeepRacer
###

<<<
English: {user_input}
>>>

Do not translate technical terms or product names.
Your output is just a direct translation of the English content, and NOTHING ELSE.
'''

prompt = PromptTemplate.from_template(template)
translation_chain = prompt | bedrock_llm_client

In [ ]:
# 对新链进行一些测试，以确保其按预期工作
inputs = [
    "Hello, my name is George",
    "Let's learn about how the reward function impacts DeepRacer model performance",
    "Important DeepRacer reward function parameters include 'speed' and 'steering_angle'",
    "What do you think about DeepRacer?"
]

for i in inputs:
    print(f'= Original text =\n {i}')
    out = translation_chain.invoke({"user_input": i})
    out = translation_format(out) # Format output nicely
    print(f'= Translation =\n {out.strip()}')

## 为 LLM 提供更新的 AWS DeepRacer 知识

Mistral Large 对 AWS DeepRacer 有一些先前的了解,但这些知识并没有与最新版本的 AWS DeepRacer 开发者指南保持同步。为了让模型了解最新的内容,我们可以创建一个利用检索增强生成(RAG)的 LangChain 链。

在本次工作坊中,我们不会详细介绍 RAG 的工作原理,更多信息请参考:
- [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)
- [Bedrock workshop](https://github.com/aws-samples/amazon-bedrock-workshop/tree/main/03_QuestionAnswering)

### AWS DeepRacer 知识库工具

这个工具是另一个专门用于回答 AWS DeepRacer 相关问题的 LangChain 链。该链使用检索增强生成(RAG)来获取与所问问题相关的文档,为回答 AWS DeepRacer 相关问题提供更好的基础。

AWS DeepRacer 开发者指南被加载并分割成块,然后转换为嵌入向量。

![](./images/Embeddings_lang.png)

嵌入是单词的向量表示,编码了语义含义。在检索增强生成中,嵌入有助于找到相关的外部知识来增强生成式聊天机器人 LLM 的能力。聊天机器人 LLM 在生成过程中使用检索到的嵌入来构建更加知识丰富和相关的响应。

![](./images/Chatbot_lang.png)

#### 设置向量存储

In [ ]:
# 从现有的Chroma向量存储中加载现有的DeepRacer信息
from langchain_chroma import Chroma

# 将预先生成的嵌入加载到向量存储中。
vectorstore_chromadb = Chroma(persist_directory="./persistent/chroma_db", embedding_function=bedrock_embeddings_client)

#### 创建 AWS DeepRacer QA 链

为了构建 RAG 链,我们使用了 LangChain 链中的一种,[Retrieval QA chain](https://python.langchain.com/docs/use_cases/question_answering/),它内置了对 RAG 的支持。

提示使用了几种技术来确保与 AWS DeepRacer 相关的响应:
- 从向量存储中检索到的文档被插入到 {context} 变量中,使其可供 LLM 使用
- 通过要求 LLM 使用通过 RAG 检索到的信息来回答问题,减少幻觉

### 构建检索链

为了确保我们能从 vector store 中检索到相关文档并生成合理的响应，我们需要：

1. 创建一个 retriever
2. 创建一个提示模板
3. 构建检索链

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(bedrock_llm_client, prompt)
retriever = vectorstore_chromadb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
# 检查链路的响应
retrieval_output = retrieval_chain.invoke({"input": "What is DeepRacer?"})

# 输出完整的响应（这也将让我们看到输入和从 vector store 中获取的上下文）
import pprint
print("\n== Full Response ==\n")
pprint.pp(retrieval_output)

# 英文答案摘要
print("\n== English Answer ==\n")
print_ww(retrieval_output['answer'].strip())

# 取答案部分，调用我们的翻译链，生成中文响应
print("\n== Chinese Answer ==\n")
chinese = translation_chain.invoke({"user_input": retrieval_output['answer']})
chinese = translation_format(chinese)
print_ww(chinese)

In [ ]:
# 我们一步一步地进行，使用我们的 Chroma 向量存储设置一个 RAG QA 链
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Given an AWS DeepRacer related question, provide a detailed answer.

<context>
{context}
</context>

Here is the human's next reply:
<human_reply>
 {question}
</human_reply>

If the answer cannot be found in the context, the AI truthfully says it does not know the answer.

Assistant:"""

prompt = PromptTemplate.from_template(prompt_template)

# 注意：我们正在使用一种较旧（已弃用）的方法来创建 QA 链和工具调用代理，
# 因为最新的 'create_tool_calling_agent' 函数截至
# 2024-05-08 还不支持 Bedrock 模型，因为缺少 .bind_tools 方法
deepracer_qa_chain = RetrievalQA.from_chain_type(
    llm=bedrock_llm_client,
    chain_type="stuff",  # 添加所有返回的文档
    retriever=vectorstore_chromadb.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3
        },  # 从向量存储中返回前 3 个相似的文档
    ),
    # return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

#### 创建 LangChain 自定义工具

现在我们将 AWS DeepRacer 问答链打包为 LangChain 自定义工具,并使其可供代理使用。

In [ ]:
from langchain.agents import Tool

# 初始化 LLM 工具
deepracer_knowledge_tool = Tool.from_function(
    name='AWS DeepRacer knowledge base',
    func=deepracer_qa_chain.run,
    description='Useful when you need to answer generic questions about AWS DeepRacer, Input: the question to answer, Output: Answer to the question',
)

In [ ]:
tools = [
        deepracer_knowledge_tool,
]

## 使用 ReAct 框架:协同语言模型中的推理和行动
大型语言模型可以交替生成对其推理的解释和特定任务的响应。

生成推理解释使模型能够推断、监控和修改行动计划,甚至处理意外情况。行动步骤允许模型与外部知识库或环境等外部资源进行交互并从中获取信息。

在这里,我们将初始化负责决定下一步行动的链。这由语言模型和提示驱动。该链的输入包括:

- 可用工具列表
- 用户输入 
- 任何先前执行的步骤(中间步骤)

然后,该链返回要执行的下一个动作,或要发送给用户的最终响应(AgentAction 或 AgentFinish)。

不同的 [LangChain 代理类型](https://python.langchain.com/docs/modules/agents/agent_types/)具有不同的推理提示风格、不同的输入编码方式以及不同的输出解析方式。

In [ ]:
from langchain.agents import initialize_agent, AgentType

react_agent = initialize_agent(tools,
                               llm=bedrock_llm_client,
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True, # displays the intermediate steps the agent performs
                               return_intermediate_steps=True,
                               #handle_parsing_errors=True,
                               )

让我们来看看这个链条使用的默认提示。

请注意,提示模板具有以下结构:
```

Answer the following questions as best you can. You have access to the following tools:

{available tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool names}
Action Input: the input to the action\nObservation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}

```

默认提示指导模型如何推理并与可用工具交互。


In [ ]:
react_agent.agent.llm_chain.prompt.template

一旦代理```react_agent```设置完成,我们就可以通过传递提示/问题来调用代理,如下所示。你可以随意调整后续调用中的提示/问题。

当运行以下命令时,你看到代理如何开始推理它需要做什么来回答问题,它使用什么工具以及它们的输入。
如果一切顺利,代理会产生一个最终答案。

下面的问题,代理通常可以在一个计划->行动->观察循环中解决,你看到一个"> Finished chain."和最终答案。

⚠️ **如果在运行下面的单元格时遇到访问被拒绝的问题,请检查Amazon Bedrock中Mixtral 8x7B和Titan Embeddings G1 – Text的状态是否都为Access granted。** ⚠️

⚠️ **如果你遇到OutputParserException。这有时会发生,因为LLM没有以正确的格式返回
问题的响应。我们将在后面的实验中通过引入few-shot提示来改进这一点,在提示中提供一些关于LLM如何响应的示例** ⚠️

In [ ]:
# 让我们尝试调用我们的新 RetrievalQA 链
# 注意：这种调用代理的方法已被废弃，但由于 LangChain 为 Amazon Bedrock 
# 提供的封装中缺少一些函数调用功能，我们无法使用 LangChain 最近版本提供的创建检索链的较新方法。
question = "What is AWS DeepRacer?"
output = react_agent(question)["output"]
print("\n== English ==\n")
print_ww(output)

# 接下来，我们使用我们的翻译链帮助我们翻译代理的输出。
chinese_output = translation_chain.invoke(output)
chinese_output = translation_format(chinese_output)
print("\n== Chinese ==\n")
print_ww(chinese_output)

## 推理链条可能存在的问题

此问题展示了使用代理时可能发生的一个问题。在解决这个问题时，代理通常会陷入一个循环，每次都是相同的思考和行动，直到它无法遵循其被指示的格式。

我们将在实验室中通过引入少数示例提示来改进这一点，提供一些示例在提示中说明 LLM 如何推理，并让代理管理解析错误。

In [ ]:
try:
    question = "How do I get started with training a DeepRacer model? anything specific I should think of?"
    output = react_agent(question)["output"]
    print(output)
except Exception as e:
    print(f"\n\nThe agent encountered an error! \n {e}")

In [ ]:
# 接下来，我们使用我们的翻译链帮助我们翻译代理的输出。
chinese_output = translation_chain.invoke(output)
chinese_output = translation_format(chinese_output)
print_ww(chinese_output)

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=bedrock_llm_client, verbose=True, memory=ConversationBufferMemory()
)

## 为 LLM 添加更多工具以提升其分析 DeepRacer 模型的能力

到目前为止,LLM 只能回答关于 AWS DeepRacer 的一般信息,这些信息是 [AWS DeepRacer 开发人员指南](https://docs.aws.amazon.com/deepracer/latest/developerguide/what-is-deepracer.html)的一部分。
在本节中,我们将把 LLM 与 AWS DeepRacer 服务 API 集成,以便 LLM 可以与您在此 AWS 账户中有权访问的不同模型进行交互。

### 创建一个列出 AWS DeepRacer 模型的工具

当代理调用此工具时,它将列出位于 AWS DeepRacer 服务中的可用 DeepRacer 模型。它已经作为一个 LangChain 自定义工具实现,并在代理调用时使用自定义代码来调用 DeepRacer 服务 API。

In [ ]:
from langchain.tools import BaseTool

class DeepRacerListModelsTool(BaseTool):
    name = "List AWS DeepRacer models"
    description = """
    Use this tool when you need to list all AWS DeepRacer models. Lists the name of each DeepRacer model.
    """

    def _run(self, input=None):
        # List all available DeepRacer models
        models = deepracer_local.list_models()
        return models

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")


### 创建一个获取 AWS DeepRacer 模型详细信息的工具

下载一个 AWS DeepRacer 模型并提取相关部分,以允许 LLM 对其训练和性能进行分析。

In [ ]:
import os
from pathlib import Path

from langchain.tools import BaseTool
from pydantic import BaseModel, Field

class ModelAnalysisInput(BaseModel):
    model_name: str = Field()

class DeepRacerModelAnalysisTool(BaseTool):
    name = "AWS DeepRacer model details"
    description = """Use this tool to get detailed information about an AWS DeepRacer model. Input: Expects the model_name as string input. Output: Python dictionary. IMPORTANT: DeepRacer track names are preceded by 'WORLD_NAME:'"""
    args_schema = ModelAnalysisInput

    def _run(self, model_name=None):
        if deepracer_local.model_exists(model_name):
            # Attempt to load model details from the local `deepracer_models` folder
            # using helpers in `utils/deepracer_local.py`
            model_data = {}
            
            model_data["model_metadata_used_for_training"] = deepracer_local.get_model_metadata(model_name)
            model_data["reward_function_used_for_training"] = deepracer_local.get_reward_function(model_name)
            model_data["hyper_parameters_used_for_training"] = deepracer_local.get_hyper_parameters(model_name)
            model_data["evaluation_results"] = deepracer_local.get_evaluation_metrics(model_name)
            
            # We exclude the training results here because they tend to be very long (and Mixtral 8x7B only has a 32K context window)
            #model_data["training_results"] = deepracer_local.get_training_metrics(model_name)
            
            model_data["evaluation_parameters"] = deepracer_local.get_evaluation_params(model_name)
            model_data["training_parameters"] = deepracer_local.get_training_params(model_name)
        
            # We hard-code the "track metadata", as the original function in the DeepRacer
            # GenAI lab simply responded with a hard-coded string
            model_data["track_meta_data"] =  "Track difficulty is an integer ranging from 100 to 1, where 1 is the hardest"
        
            return model_data
        else:
            return f"Model with name {model_name} does not exist."

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")


在创建了第三个也是最后一个工具后,我们将这 3 个工具添加到一个列表中,该列表将作为位置参数传递给代理初始化函数。这使得代理在解析提示时可以使用这些工具。

In [ ]:
tools = [
        DeepRacerModelAnalysisTool(),
        DeepRacerListModelsTool(),
        deepracer_knowledge_tool,
]

In [ ]:
from langchain.agents import initialize_agent, AgentType

react_agent = initialize_agent(tools,
                               llm=bedrock_llm_client,
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True, # verbose logs is turned on to give insights into how the agent will reason and use the tools
                               return_intermediate_steps=True,
                               handle_parsing_errors=True,
                               )

In [ ]:
import json

prompt_template = """Human: Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above.
Action: the action to take, should be one of {tools}.
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

<examples>
<example>
Question: What is AWS DeepRacer?
Thought: I need to get more information on what AWS DeepRacer is, I should check the AWS DeepRacer knowledge base
Action: AWS DeepRacer knowledge base
Action Input: What is AWS DeepRacer?
Observation:  AWS DeepRacer is the fastest way to get rolling with reinforcement learning (RL), literally, with a fully autonomous 1/18th scale race car driven by reinforcement learning, 3D racing simulator, and a global racing league.
Final Answer: AWS DeepRacer is the fastest way to get rolling with reinforcement learning (RL), literally, with a fully autonomous 1/18th scale race car driven by reinforcement learning, 3D racing simulator, and a global racing league.
</example>
<example>
Question: how can I improve my-deepracer-model?
Thought: I need to check the details and performance of the AWS DeepRacer model to answer this question.
Action: AWS DeepRacer model details
Action Input: my-deepracer-model
Observation: ....
Thought: I should check the AWS DeepRacer knowledge base on how to improve a DeepRacer model
Action: AWS DeepRacer knowledge base
Action Input: Improve deepracer model
Observation: ....
Thought: With the details about the model and knowledge on how to improve a model I know have enough information.
Final Answer: The my-deepracer-model model is designed and trained to follow the center line of the track. The model details, reward function, and hyperparameters used for training provide evidence that this model prioritizes staying on the center line, making it a good performing model for that task.
</example>
</examples>

You are an AWS DeepRacer Q&A bot, give a detailed answer to the question.
You can only answer question not perform any changes to any DeepRacer models.
Question: {input}


Assistant:
{agent_scratchpad}"""

# Create a prompt template and inject the names of all available tools in the {tools} variable. This can be done using partial_variables
tool_names = json.dumps([d.name for d in tools])
template = PromptTemplate(
    template=prompt_template,
    input_variables=["input", "agent_scratchpad"],
    partial_variables={"tools": tool_names},
)
react_agent.agent.llm_chain.prompt = template


通过添加 DeepRacerListModelsTool 和 DeepRacerModelAnalysisTool 自定义工具,代理现在能够:
1. 列出可用的 AWS DeepRacer 模型
2. 下载指定的 DeepRacer 模型并提取:
     - 训练数据 
     - 评估数据
     - 提取模型元数据,如动作空间、超参数等
     - 提取奖励函数

正如我们在实验室前面所说,代理可以访问此 AWS 账户中的一些 DeepRacer 模型。

In [ ]:
for model in models:
    print(f"Name: {model}")

在向代理提问时可以使用这些 DeepRacer 模型名称。

在这个问题中,代理将使用 **List AWS DeepRacer models** 工具通过 DeepRacer 服务 API 获取当前 AWS 账户中 DeepRacer 模型的名称。在 **Finished chain** 之后是返回给用户的输出。

在 **Finished chain** 之前看到的文本只是由于我们为代理使用的 LLM 打开了 **verbose=True** 而产生的日志。这是为了更好地突出代理如何推理和与工具交互。

In [ ]:
question = "What are the names of DeepRacer models I have access to?"
output = react_agent(question)["output"]

print('\n== English ==\n')
print_ww(output) # Print the model's output

chinese_output = translation_chain.invoke({"user_input": output})
chinese_output = translation_format(chinese_output) # Format output nicely
print('\n== Chinese ==\n')
print_ww(chinese_output)

## 下一步：考虑模型细节

在这个问题中,代理将使用 **AWS DeepRacer model details** 工具通过 DeepRacer 服务 API 提取 **MODEL_NAME** 模型详细信息(超参数、奖励函数、训练指标、评估指标和使用的赛道)。然后将这些数据作为 json 返回给代理,代理将搜索评估使用的是哪个赛道。

在 **Finished chain** 之后是返回给用户的输出。

将 MODEL_NAME 变量更改为您想要了解的模型的名称。

In [ ]:
MODEL_NAME="AtoZ-CCW-Centerline"
COMPARE_MODEL_NAME="AtoZ-CCW-Steering-Penalty"

In [ ]:
question = "Which track was " + MODEL_NAME + " trained and evaluated on?"
output = react_agent(question)["output"]

print('\n== English ==\n')
print_ww(output) # Print the model's output

chinese_output = translation_chain.invoke({"user_input": output})
chinese_output = translation_format(chinese_output) # Format output nicely
print('\n== Chinese ==\n')
print_ww(chinese_output)

## 让我们尝试更多的问题,看看代理如何推理和使用工具。

In [ ]:
question = "Is " + MODEL_NAME + " or " + COMPARE_MODEL_NAME + " the better model? please also state why"
output = react_agent(question)["output"]

print('\n== English ==\n')
print_ww(output) # Print the model's output

chinese_output = translation_chain.invoke({"user_input": output})
chinese_output = translation_format(chinese_output) # Format output nicely
print('\n== Chinese ==\n')
print_ww(chinese_output)

In [ ]:
question = "How can I improve the training for " + MODEL_NAME + "?"
output = react_agent(question)["output"]

print('\n== English ==\n')
print_ww(output) # Print the model's output

chinese_output = translation_chain.invoke({"user_input": output})
chinese_output = translation_format(chinese_output) # Format output nicely
print('\n== Chinese ==\n')
print_ww(chinese_output)

In [ ]:
question = "Should I change any of the hyperparameters that was used to train " + MODEL_NAME + " to improve the training?"
output = react_agent(question)["output"]

print('\n== English ==\n')
print_ww(output) # Print the model's output

chinese_output = translation_chain.invoke({"user_input": output})
chinese_output = translation_format(chinese_output) # Format output nicely
print('\n== Chinese ==\n')
print_ww(chinese_output)

In [ ]:
question = "Write a simple deepracer reward function for me"
output = react_agent(question)["output"]

print('\n== English ==\n')
print_ww(output) # Print the model's output

# NOTE: We disable the formatter here because the characters it searches for may be 
# present in any Python code the model generates
chinese_output = translation_chain.invoke({"user_input": output})
#chinese_output = translation_format(chinese_output) # Format output nicely
print('\n== Chinese ==\n')
print_ww(chinese_output)

# 回顾

在本实验中,我们使用自定义工具将 LangChain 代理与 AWS DeepRacer 服务集成在一起。这使得 LLM 能够帮助我们进行模型分析以及如何改进模型。


### 后续步骤:
[Amazon Bedrock 研讨会](https://github.com/aws-samples/amazon-bedrock-workshop) - 更多使用案例和使用 Amazon Bedrock 构建的方法

[SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html) - 为各种问题类型提供预训练的开源模型,帮助您开始机器学习。